# Homework 2 - TF-IDF Classifier

Ваша цель обучить классификатор который будет находить "токсичные" комментарии и опубликовать решения на Kaggle [Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)

В процессе обучения нужно ответить на ***[вопросы](https://docs.google.com/forms/d/e/1FAIpQLSd9mQx8EFpSH6FhCy1M_FmISzy3lhgyyqV3TN0pmtop7slmTA/viewform?usp=sf_link)***

Данные можно скачать тут - https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data



In [1]:
import numpy as np
import pandas as pd
import gc
from scipy.sparse import hstack
from os import cpu_count

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('D:/STUDY/Data_Science_Study/University_course/HW_2/train.csv').fillna(' ')
test = pd.read_csv('D:/STUDY/Data_Science_Study/University_course/HW_2/test.csv').fillna(' ')

Стадартными подходами для анализа текста являются [Bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) и его модификация [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

Они реалзованны в `sklearn` в виде [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) и [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

Более подробней про них можно посмотреть [тут](https://github.com/udsclub/workshop/blob/master/notebooks/UDS-workshop-feature-extraction-and-engineering.ipynb)

In [3]:
train_text = train['comment_text']
test_text = test['comment_text']
#all_text = pd.concat([train_text, test_text])

# Data cleaning

In [4]:
import re

In [5]:
def cleaning_data(noizy_comment):
    noizy_comment= re.sub(r'http\S+', '', noizy_comment)
    noizy_comment = noizy_comment.lower()
    noizy_comment = re.sub(r"what's", "what is ", noizy_comment)
    noizy_comment = re.sub(r"\'s", " ", noizy_comment)
    noizy_comment = re.sub(r"\'ve", " have ", noizy_comment)
    noizy_comment = re.sub(r"can't", "cannot ", noizy_comment)
    noizy_comment = re.sub(r"n't", " not ", noizy_comment)
    noizy_comment = re.sub(r"i'm", "i am ", noizy_comment)
    noizy_comment = re.sub(r"\'re", " are ", noizy_comment)
    noizy_comment = re.sub(r"\'d", " would ", noizy_comment)
    noizy_comment = re.sub(r"\'ll", " will ", noizy_comment)
    noizy_comment = re.sub(r"\'scuse", " excuse ", noizy_comment)
    noizy_comment = re.sub(r'\W', ' ', noizy_comment)
    noizy_comment = re.sub(r'\s+', ' ', noizy_comment)
    noizy_comment = re.sub(' +',' ',noizy_comment)
    noizy_comment = re.sub(r'\n','',noizy_comment)
    noizy_comment = noizy_comment.strip(' ')
    cleaned_comment = noizy_comment
    return cleaned_comment

In [6]:
def data_set_cleaning(noizy_data_set):
    cleaned_text = []
    for i in range(0,len(noizy_data_set)):
        text_cleaning = cleaning_data(noizy_data_set[i])
        cleaned_text.append(text_cleaning)
    noizy_data_set = pd.Series(cleaned_text).astype(str)
    return noizy_data_set

In [7]:
train_text = data_set_cleaning(train_text)

In [8]:
test_text = data_set_cleaning(test_text)

In [9]:
all_text = pd.concat([train_text, test_text])

# Preprocessing

In [ ]:
# Попробуйте разные Vectorizer и разные размеры n-gramm, стоп-слова, обрезку редких слов, обрезку слишком частых слов
word_vectorizer = ... # TfidfVectorizer или CountVectorizer

In [10]:
word_vectorizer = TfidfVectorizer(
    max_df=0.85,
    min_df=3,
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)


In [11]:
word_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.85, max_features=10000, min_df=3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [12]:
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [14]:
char_vectorizer = TfidfVectorizer(
    max_df=0.85,
    min_df=3,
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    #ngram_range=(2, 4),
    ngram_range=(2, 5),
    max_features=30000)

In [15]:
char_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.85, max_features=30000, min_df=3,
        ngram_range=(2, 5), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [16]:
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [18]:
test_features = hstack([test_word_features, test_char_features])#.tocsr()

In [19]:
del test_word_features
del test_char_features

In [20]:
gc.collect()

0

In [21]:
train_features = hstack([train_word_features, train_char_features])#.tocsr()

In [22]:
del test_text
del train_text
del train_word_features
del train_char_features

In [23]:
gc.collect()

0

In [21]:
#word_vectorizer.fit(all_text)
#train_word_features = word_vectorizer.transform(train_text)
#test_word_features = word_vectorizer.transform(test_text)

Для классификации будем использовать логистическую регрессию [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

In [24]:
classifier = LogisticRegression(class_weight="balanced",solver="lbfgs") # Попробуйте разные параметры, найтдите оттимальные на кросс-валидации

Будем тренировать по одному классификатору на каждый класс. 

Что бы провалидировать качество модели воспользуемся функцией [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)

In [25]:
n_jobs = max(cpu_count()-1, 1)

In [ ]:
scores= []

for class_name in class_names:
    train_target = train[class_name]

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, scoring='roc_auc',n_jobs=n_jobs))
    
    print('CV score for class {} is {}'.format(class_name, cv_score))
    scores.append(cv_score)

print('Total score is {}'.format(np.mean(scores)))

In [28]:
scores= []

for class_name in class_names:
    train_target = train[class_name]

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, scoring='roc_auc',n_jobs=n_jobs))
    
    print('CV score for class {} is {}'.format(class_name, cv_score))
    scores.append(cv_score)

print('Total score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9773823478427864
CV score for class severe_toxic is 0.9884226506149122
CV score for class obscene is 0.990933908529361
CV score for class threat is 0.9899151786125492
CV score for class insult is 0.9827306393094002
CV score for class identity_hate is 0.9820679450089799
Total score is 0.9852421116529982


In [ ]:
CV score for class toxic is 0.9763836010389947
CV score for class severe_toxic is 0.9863922140280357
CV score for class obscene is 0.9888869666383201
CV score for class threat is 0.9858681687290457
CV score for class insult is 0.9807645145826589
CV score for class identity_hate is 0.9808867032410958
Total score is 0.983197028043025

In [27]:
scores= []

for class_name in class_names:
    train_target = train[class_name]

    cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, scoring='roc_auc'))
    
    print('CV score for class {} is {}'.format(class_name, cv_score))
    scores.append(cv_score)

print('Total score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9685394804763826
CV score for class severe_toxic is 0.9831972794553816
CV score for class obscene is 0.981616766505934
CV score for class threat is 0.9822578771961118
CV score for class insult is 0.9741350581893133
CV score for class identity_hate is 0.9713647943992112
Total score is 0.9768518760370558


Попробуйте подобрать лучшие параметры для `word_vectorizer` и `classifier` оптимизируя метрику [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)


---

Опубликуйте лучшие решение на [Kaggle Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/submit)

In [29]:
submission = pd.DataFrame.from_dict({'id': test['id']})

In [30]:
for class_name in class_names:
    train_target = train[class_name]
    x_test = train_features
    y_test = train_target[:]
    print("classifier",class_name)
    classifier.fit(x_test, y_test)
    print("submission",class_name)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1] 
    print("end",class_name)
    #classifier.fit(...)
    #...
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

classifier toxic
submission toxic
end toxic
classifier severe_toxic
submission severe_toxic
end severe_toxic
classifier obscene
submission obscene
end obscene
classifier threat
submission threat
end threat
classifier insult
submission insult
end insult
classifier identity_hate
submission identity_hate
end identity_hate


In [31]:
submission.to_csv('submission.csv', index=False)